In [72]:
import torch.nn as nn
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import sys
from torch.utils.data import TensorDataset, DataLoader
import re
from pymorphy3 import MorphAnalyzer
from nltk.corpus import stopwords
from razdel import tokenize
from navec import Navec

In [73]:
device = torch.device ( "cuda:0" if torch.cuda.is_available() else "cpu" )
print ( device )

cuda:0


In [74]:
df = pd.read_csv("../datasets/Petitions.csv")
corpus = df.sample(10000, random_state=42)
corpus

,id,public_petition_text,reason_category
1638,3315301,На газоне разбросан различный мусор. \nПросьба...,Благоустройство
53659,3372310,"Конструкции, препятствующие парковке",Благоустройство
25910,3294518,Лестницу убирают раз в три месяца и просто выл...,Содержание МКД
52453,3158213,мусор с задней стороны дома,Благоустройство
12590,3356661,Снова не работает уличный фонарь у входной две...,Содержание МКД
...,...,...,...
33876,3084944,Убрать листву с газона.,Благоустройство
6550,3344824,При входе в первый подъезд на первом этаже ест...,Содержание МКД
16469,3079420,"Прошу убрать резиновые покрышки, которые в соо...",Благоустройство
35789,2964425,Мусор возле магазина магнит,Благоустройство


In [75]:
corpus_text = corpus["public_petition_text"]
corpus_text_category = corpus["reason_category"]

In [76]:
print(corpus_text)
print(type(corpus_text))

1638     На газоне разбросан различный мусор. \nПросьба...
53659                 Конструкции, препятствующие парковке
25910    Лестницу убирают раз в три месяца и просто выл...
52453                          мусор с задней стороны дома
12590    Снова не работает уличный фонарь у входной две...
                               ...                        
33876                              Убрать листву с газона.
6550     При входе в первый подъезд на первом этаже ест...
16469    Прошу убрать резиновые покрышки, которые в соо...
35789                          Мусор возле магазина магнит
44639       1 парадная, требуется отремонтировать доводчик
Name: public_petition_text, Length: 10000, dtype: object
<class 'pandas.core.series.Series'>


In [77]:
from sklearn.preprocessing import LabelEncoder

In [78]:
le = LabelEncoder()
corpus_text_category_encoded = le.fit_transform(corpus_text_category)
dict(zip(range(0, len(set(corpus_text_category_encoded))),le.inverse_transform(list(set(corpus_text_category_encoded)))))

{0: 'Благоустройство',
 1: 'Водоотведение',
 2: 'Водоснабжение',
 3: 'Кровля',
 4: 'Нарушение порядка пользования общим имуществом',
 5: 'Нарушение правил пользования общим имуществом',
 6: 'Незаконная информационная и (или) рекламная конструкция',
 7: 'Незаконная реализация товаров с торгового оборудования (прилавок, ящик, с земли)',
 8: 'Повреждения или неисправность элементов уличной инфраструктуры',
 9: 'Подвалы',
 10: 'Санитарное состояние',
 11: 'Содержание МКД',
 12: 'Состояние рекламных или информационных конструкций',
 13: 'Фасад',
 14: 'Центральное отопление'}

In [79]:
corpus_encoded = []
for i in range(len(corpus_text)):
    corpus_encoded.append((corpus_text.iloc[i], corpus_text_category_encoded[i]))

In [80]:
navec = Navec.load('../models/navec_hudlit_v1_12B_500K_300d_100q.tar')

In [81]:
from nltk.corpus import stopwords
def prepare_data(corpus):
    morph = MorphAnalyzer()
    from nltk.corpus import stopwords
    stopwords = stopwords.words('russian')
    prepared_data = []
    for petition, label in corpus:   
        data = petition.lower()
        data1 = re.sub('[\\r|\\n]+', ' ', data)
        data2 = re.sub('[a-zA-Z]+', '', data1)
        data3 = re.sub('[0-9]+', '', data2)
        data4 = re.sub('[^\s^\w]+', '', data3)
        data5 = list(tokenize(data4))
        data6 = map(lambda x: x.text , data5)
        data7 = map(lambda x: morph.normal_forms(x)[0], data6)
        data8 = [w for w in data7 if w not in stopwords]
        data9 = [navec[x].tolist() for x in data8 if x in navec]
        if(len(data9)):
            prepared_data.append((torch.tensor(data9).type(torch.float32), label))
    return prepared_data

In [82]:
corpus_text_prep = prepare_data(corpus_encoded)

In [83]:
from torch.nn.utils.rnn import pad_sequence
corpus_text_padded = pad_sequence([tupl[0] for tupl in corpus_text_prep], True)

In [84]:
corpus_text_padded.shape

torch.Size([9965, 229, 300])

In [85]:
X_train, X_test, y_train, y_test = train_test_split(corpus_text_padded, [tupl[1] for tupl in corpus_text_prep], test_size=0.1, stratify=[tupl[1] for tupl in corpus_text_prep])


In [86]:
ds = TensorDataset(X_train, torch.tensor(y_train).type(torch.int64))
dl = DataLoader(ds, batch_size=32, shuffle=True)

In [87]:
class Rnn(nn.Module):
    def __init__(self):
        super(Rnn, self).__init__()
        self.rnn = nn.RNN(300, 128, batch_first=True)
        self.first_linear = nn.Linear(128, 15)
    def forward(self, x):
        y = self.rnn(x)[1]
        #y = self.first_linear(torch.index_select(y, 1, torch.tensor(y.shape[1]-1).to(device)))
        y = self.first_linear(y[-1])
        return y

In [88]:
loss = nn.CrossEntropyLoss()
model = Rnn()
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [89]:
epochs = 10
for epoch in range(epochs):
    sum_loss = 0
    for x_b, y_b in dl:
        x_b = x_b.to(device)
        y_b = y_b.to(device)
        optimizer.zero_grad()
        outputs = model(x_b)
        #outputs= outputs.view(x_b.shape[0],15)
        loss_value = loss(outputs, y_b)
        sum_loss += loss_value
        loss_value.backward()
        optimizer.step()
        x_b = x_b.to("cpu")
        y_b = y_b.to("cpu")
        
        #sys.stderr.write(f'Батч {batch + 1}/{len(dl)}, Значение функции потерь: {loss_value.item()}\r')

    print(f'Эпоха {epoch + 1}, Значение функции потерь: {sum_loss/len(dl)}')

Эпоха 1, Значение функции потерь: 1.4274958372116089
Эпоха 2, Значение функции потерь: 1.575461983680725
Эпоха 3, Значение функции потерь: 1.4686797857284546
Эпоха 4, Значение функции потерь: 1.3945531845092773
Эпоха 5, Значение функции потерь: 1.3837037086486816
Эпоха 6, Значение функции потерь: 1.379629373550415
Эпоха 7, Значение функции потерь: 1.3802326917648315
Эпоха 8, Значение функции потерь: 1.3870748281478882
Эпоха 9, Значение функции потерь: 1.3860352039337158
Эпоха 10, Значение функции потерь: 1.3816720247268677


In [90]:
model.to("cpu")
answ = model.forward(X_test)

In [91]:
answ_encoded = []
for i in answ:
    answ_encoded.append(torch.argmax(i).to("cpu").item())

In [92]:
from sklearn.metrics import classification_report
print(classification_report(y_test, answ_encoded))

              precision    recall  f1-score   support

           0       0.59      0.97      0.73       582
           1       0.00      0.00      0.00         4
           2       0.00      0.00      0.00        13
           3       0.00      0.00      0.00        15
           4       0.00      0.00      0.00         4
           5       0.00      0.00      0.00        32
           6       0.00      0.00      0.00        32
           7       0.00      0.00      0.00         4
           8       0.00      0.00      0.00        19
           9       0.00      0.00      0.00         4
          10       0.00      0.00      0.00         7
          11       0.39      0.05      0.09       241
          12       0.00      0.00      0.00        12
          13       0.00      0.00      0.00        23
          14       0.00      0.00      0.00         5

    accuracy                           0.58       997
   macro avg       0.07      0.07      0.06       997
weighted avg       0.44   

c:\Users\XE\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\XE\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\XE\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

In [94]:
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.rnn = nn.LSTM(300, 64, batch_first = True)
        self.first_linear = nn.Linear(64, 15)
        #self.first_linear = nn.Linear(64, 32)
        #self.first_activ = nn.LeakyReLU()
        #self.second_linear = nn.Linear(32, 15)
    def forward(self, x):
        y = self.rnn(x)[1]
        #y = self.first_linear(torch.index_select(y, 1, torch.tensor(y.shape[1]-1).to(device)))
        y = self.first_linear(y[-1])
        #y = self.first_activ(y)
        #y = self.second_linear(y)
        return y

In [95]:
loss1 = nn.CrossEntropyLoss()
model1 = LSTM()
model1.to(device)
optimizer1 = torch.optim.Adam(model.parameters(), lr=0.0025)

In [101]:
epochs = 10
for epoch in range(epochs):
    sum_loss = 0
    for x_b, y_b in dl:
        x_b = x_b.to(device)
        y_b = y_b.to(device)
        optimizer1.zero_grad()
        outputs = model1(x_b)
        outputs= outputs.view(x_b.shape[0],15)
        loss_value = loss1(outputs, y_b)
        loss_value.backward()
        optimizer1.step()
        sum_loss += loss_value
        x_b = x_b.to("cpu")
        y_b = y_b.to("cpu")
        
        #sys.stderr.write(f'Батч {batch + 1}/{len(dl)}, Значение функции потерь: {loss_value.item()}\r')

    print(f'Эпоха {epoch + 1}, Значение функции потерь: {sum_loss/len(dl)}')

Эпоха 1, Значение функции потерь: 2.654219388961792
Эпоха 2, Значение функции потерь: 2.6541669368743896
Эпоха 3, Значение функции потерь: 2.6542744636535645
Эпоха 4, Значение функции потерь: 2.6542510986328125
Эпоха 5, Значение функции потерь: 2.6542344093322754
Эпоха 6, Значение функции потерь: 2.654247999191284
Эпоха 7, Значение функции потерь: 2.654240846633911
Эпоха 8, Значение функции потерь: 2.6542809009552
Эпоха 9, Значение функции потерь: 2.6542749404907227
Эпоха 10, Значение функции потерь: 2.654237985610962


In [119]:
model1.to("cpu")
answ1 = model1.forward(X_test)

RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 3330084864 bytes.

In [122]:
answ_encoded1 = []
for i in answ1:
    answ_encoded1.append(torch.argmax(i).item())

In [123]:
from sklearn.metrics import classification_report
print(classification_report(y_test, answ_encoded1))

ValueError: Found input variables with inconsistent numbers of samples: [997, 1]

In [125]:
class GRU(nn.Module):
    def __init__(self):
        super(GRU, self).__init__()
        self.rnn = nn.GRU(300, 64, batch_first = True)
        self.first_linear = nn.Linear(64, 15)
        #self.first_linear = nn.Linear(64, 32)
        #self.first_activ = nn.LeakyReLU()
        #self.second_linear = nn.Linear(32, 15)
    def forward(self, x):
        y = self.rnn(x)[1]
        #y = self.first_linear(torch.index_select(y, 1, torch.tensor(y.shape[1]-1).to(device)))
        y = self.first_linear(y[-1])
        #y = self.first_activ(y)
        #y = self.second_linear(y)
        return y

In [126]:
loss2 = nn.CrossEntropyLoss()
model2 = GRU()
model2.to(device)
optimizer2 = torch.optim.Adam(model.parameters(), lr=0.0025)

In [127]:
epochs = 10
for epoch in range(epochs):
    batch = 0
    for x_b, y_b in dl:
        x_b = x_b.to(device)
        y_b = y_b.to(device)
        optimizer2.zero_grad()
        batch+=1
        outputs = model2(x_b)
        #outputs= outputs.view(x_b.shape[0],15)
        loss_value = loss2(outputs, y_b)
        loss_value.backward()
        
        optimizer2.step()
        
        #sys.stderr.write(f'Батч {batch + 1}/{len(dl)}, Значение функции потерь: {loss_value.item()}\r')

    print(f'Эпоха {epoch + 1}, Значение функции потерь: {loss_value.item()}')

Эпоха 1, Значение функции потерь: 2.737246513366699
Эпоха 2, Значение функции потерь: 2.7423622608184814
Эпоха 3, Значение функции потерь: 2.7218260765075684
Эпоха 4, Значение функции потерь: 2.757697820663452
Эпоха 5, Значение функции потерь: 2.802489995956421
Эпоха 6, Значение функции потерь: 2.750302791595459
Эпоха 7, Значение функции потерь: 2.762470006942749
Эпоха 8, Значение функции потерь: 2.775080919265747
Эпоха 9, Значение функции потерь: 2.7875592708587646
Эпоха 10, Значение функции потерь: 2.757697820663452


In [128]:
answ2 = model2.forward(X_test.to(device))
answ_encoded2 = []
for i in answ2:
    answ_encoded2.append(torch.argmax(i).to("cpu").item())
from sklearn.metrics import classification_report
print(classification_report(y_test, answ_encoded2))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       582
           1       0.00      0.00      0.00         4
           2       0.00      0.00      0.00        13
           3       0.00      0.00      0.00        15
           4       0.00      0.00      0.00         4
           5       0.00      0.00      0.00        32
           6       0.00      0.00      0.00        32
           7       0.00      0.00      0.00         4
           8       0.00      0.00      0.00        19
           9       0.00      0.00      0.00         4
          10       0.01      1.00      0.01         7
          11       0.00      0.00      0.00       241
          12       0.00      0.00      0.00        12
          13       0.00      0.00      0.00        23
          14       0.00      0.00      0.00         5

    accuracy                           0.01       997
   macro avg       0.00      0.07      0.00       997
weighted avg       0.00   

c:\Users\XE\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\XE\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\XE\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier